## Tests of the Heat Pump Calculator API
Mostly changing individual inputs to investigate effects on results.

This assumes that the FastAPI server is running either locally or remotely,
as this code makes HTTP requests to the server.

The requirements.txt file for this project does not include the packages
necessary to run this notebook (e.g. ipykernel, requests, dotmap).  Run this code
from a Python kernel that does have the necessary packages.

In [1]:
import requests as req
from dotmap import DotMap as DM
import pandas as pd

url = 'http://localhost:8000'

def post(endpoint: str, data: dict | DM):

    if type(data) is DM:
        json_data = data.toDict()
    else:
        json_data = data
    return DM(req.post(url + endpoint, json=json_data).json())

def summary(res):
    """Prints key results from heat pump analysis.
    """
    print(f"""
--- Fuel / Electricity Changes ---
Fuel Savings: {-res.change_detail.annual_results.fuel_units:,.3g} units
Electricity Increase: {res.change_detail.annual_results.all_kwh:,.0f} kWh
Peak Demand Increase, kW: {res.change_detail.annual_results.all_kw_max:.2f} kW
Fuel $ Savings, 1st year: $ {-res.change_detail.annual_results.fuel_dol:,.0f}
Electricity $ Increase, 1st year: $ {res.change_detail.annual_results.all_elec_dol:,.0f}
Net 1st year Savings: $ {-res.change_detail.annual_results.total_dol:,.0f}

--- Heat Pump Performance ---
Load Served: {res.with_heat_pump_detail.annual_results.hp_load_mmbtu:,.1f} MMBTU
Load Fraction: {res.with_heat_pump_detail.annual_results.hp_load_frac:.3f}
Fraction Capacity Used: {res.with_heat_pump_detail.annual_results.hp_capacity_used_max:.2f}
Annual COP: {res.with_heat_pump_detail.annual_results.cop:.2f}
CO2 Saved: {res.misc.co2_lbs_saved:,.0f} pounds/year

--- Financial Results ---
Benefit/Cost Ratio: {res.financial.bc_ratio:.2f}
Internal Rate of Return: {res.financial.irr * 100:.1f}%
Discounted Payback: {res.financial.discounted_payback:.1f} years
Simple Payback: {res.financial.simple_payback:.1f} years
Incremental Fuel Cost: $ {res.misc.fuel_price_incremental:.4g} / unit
Incremental Electricty Cost: $ {res.misc.elec_rate_incremental:.3f} / kWh

--- Building Thermal Characteristics ---
Design Heat Load: {res.base_case_detail.design_heat_load:,.0f} BTU/hour
Design Heating Temperature: {res.base_case_detail.design_heat_temp:.1f} deg F
UA True-up Multiplier: {res.misc.ua_true_up:.2f}
""")

In [16]:
base_inp = DM(
    {
    "bldg_name": "Test Building",
    "notes": "Notes about the analysis.",
	"bldg_model_inputs": {
		"city_id": 1,
		"heat_pump": {
			"hspf_type": "hspf",
			"hspf": 13.25,
			"max_out_5f": 15000,
			"low_temp_cutoff": 5,
			"off_months": None,
			"frac_exposed_to_hp": 0.4,
			"frac_adjacent_to_hp": 0.25,
			"doors_open_to_adjacent": True,
			"bedroom_temp_tolerance": "med",
			"serves_garage": False
		},
		"exist_heat_system": {
			"heat_fuel_id": 4,
			"heating_effic": 0.8,
			"aux_elec_use": 3.0,
			"serves_dhw": False,
			"serves_clothes_drying": False,
			"serves_cooking": False,
			"occupant_count": 2.3
		},
        "building_type": "residential",
		"bldg_floor_area": 3600,
		"garage_stall_count": 2,
		"indoor_heat_setpoint": 70,
		"insul_level": "wall2x6plus"
	},
	"heat_pump_cost": {
		"capital_cost": 4500,
		"rebate_amount": 1000,
        "heat_pump_life": 14,
		"op_cost_chg": -50,
		"frac_financed": 0.0,
		"loan_term": 10,
		"loan_interest": 0.05
	},
	"economic_inputs": {
		"utility_id": 1,
        "pce_limit": 750,
        "elec_rate_forecast": 0.023,
        "elec_rate_override": None,
        "pce_rate_override": None,
        "customer_charge_override": None,
        "co2_lbs_per_kwh_override": None,
        "fuel_price_override": None,
        "fuel_price_forecast": 0.033,
        "sales_tax_override": None,
        "discount_rate": 0.0537,
        "inflation_rate": 0.023
	},
	"actual_fuel_use": {
		"secondary_fuel_units": 1200.0,
        "annual_electric_is_just_space_heat": False,
        "electric_use_by_month": None
	}
})

In [17]:
res = post('/heat/analyze-heat-pump', base_inp)
res.pprint()

{'base_case_detail': {'annual_results': {'all_elec_dol': 1546.4412073342078,
                                         'all_kw_max': 2.29368934703218,
                                         'all_kwh': 6565.84571588275,
                                         'cop': None,
                                         'fuel_dol': 3955.6523118039186,
                                         'fuel_units': 1198.6825187284603,
                                         'hp_capacity_used_max': 0.0,
                                         'hp_kw_max': 0.0,
                                         'hp_kwh': 0.0,
                                         'hp_load_frac': 0.0,
                                         'hp_load_mmbtu': 0.0,
                                         'period': 'Annual',
                                         'secondary_fuel_mmbtu': 164.7613095642643,
                                         'secondary_fuel_units': 1198.6825187284603,
                                      

In [19]:
summary(res)


--- Fuel / Electricity Changes ---
Fuel Savings: 585 units
Electricity Increase: 7,679 kWh
Peak Demand Increase, kW: 2.35 kW
Fuel $ Savings, 1st year: $ 1,932
Electricity $ Increase, 1st year: $ 1,696
Net 1st year Savings: $ 236

--- Heat Pump Performance ---
Load Served: 65.0 MMBTU
Load Fraction: 0.491
Fraction Capacity Used: 1.00
Annual COP: 2.42
CO2 Saved: 7,166 pounds/year

--- Financial Results ---
Benefit/Cost Ratio: 1.27
Internal Rate of Return: 8.8%
Discounted Payback: 11.3 years
Simple Payback: 8.8 years
Incremental Fuel Cost: $ 3.3 / unit
Incremental Electricty Cost: $ 0.221 / kWh

--- Building Thermal Characteristics ---
Design Heat Load: 52,047 BTU/hour
Design Heating Temperature: -4.5 deg F
UA True-up Multiplier: 1.22



### Sensitivity Tests

In [24]:
pd.options.display.float_format = '{:.4g}'.format

def sens(base_input, inp_key: str, output_cols, frac_chg=0.2, test_values=None):
    inp = base_input.copy()
    inp_prop = f'inp.{inp_key}'
    if test_values:
        inp_vals = test_values
    else:
        base_val = eval(inp_prop)
        inp_vals = (base_val * (1. - frac_chg), base_val, base_val * (1. + frac_chg))

    data = {}
    for col in output_cols:
        data[col] = []

    for ival in inp_vals:
        if type(ival) is str:
            exec(f'{inp_prop} = "{ival}"')
        else:
            exec(f'{inp_prop} = {ival}')
        res = post('/heat/analyze-heat-pump', inp)
        for col in output_cols:
            data[col].append(eval(f'res.{col}'))

    df = pd.DataFrame(index=inp_vals, data=data)
    df.index.name = inp_key.split('.')[-1]

    return df


In [25]:
sens(
    base_inp,
    'bldg_model_inputs.heat_pump.hspf', 
    ('with_heat_pump_detail.annual_results.cop', 'change_detail.annual_results.total_dol')
)

,with_heat_pump_detail.annual_results.cop,change_detail.annual_results.total_dol
hspf,,
10.6,2.166,-30.41
13.25,2.421,-235.7
15.9,2.652,-387.2


In [26]:
mod_inp = base_inp.copy()
mod_inp.bldg_model_inputs.heat_pump.hspf = 11.0
sens(
    mod_inp,
    'bldg_model_inputs.heat_pump.hspf_type', 
    ('with_heat_pump_detail.annual_results.cop', 'change_detail.annual_results.total_dol'),
    test_values = ('hspf', 'hspf2_reg4', 'hspf2_reg5')
)

,with_heat_pump_detail.annual_results.cop,change_detail.annual_results.total_dol
hspf_type,,
hspf,2.206,-66.09
hspf2_reg4,2.29,-135.6
hspf2_reg5,2.441,-249.7


In [31]:
mod_inp = base_inp.copy()
mod_inp.bldg_model_inputs.heat_pump.max_out_5f = 15000.0
sens(
    mod_inp,
    'bldg_model_inputs.heat_pump.max_out_5f', 
    ('with_heat_pump_detail.annual_results.hp_capacity_used_max', 
     'with_heat_pump_detail.annual_results.hp_load_frac', 
     'with_heat_pump_detail.annual_results.cop')
)

,with_heat_pump_detail.annual_results.hp_capacity_used_max,with_heat_pump_detail.annual_results.hp_load_frac,with_heat_pump_detail.annual_results.cop
max_out_5f,,,
1.2e+04,1,0.4669,2.441
1.5e+04,1,0.491,2.421
1.8e+04,1,0.4981,2.414


In [32]:
sens(
    base_inp,
    'bldg_model_inputs.heat_pump.frac_exposed_to_hp', 
    ('with_heat_pump_detail.annual_results.hp_load_frac', 'with_heat_pump_detail.annual_results.hp_capacity_used_max')
)

,with_heat_pump_detail.annual_results.hp_load_frac,with_heat_pump_detail.annual_results.hp_capacity_used_max
frac_exposed_to_hp,,
0.32,0.436,1
0.4,0.491,1
0.48,0.5397,1


In [60]:
mod_inp = base_inp.copy()
mod_inp.bldg_model_inputs.heat_pump.max_out_5f = 25000.0
mod_inp.bldg_model_inputs.heat_pump.bedroom_temp_tolerance = 'low'
sens(
    mod_inp,
    'bldg_model_inputs.heat_pump.doors_open_to_adjacent', 
    ('with_heat_pump_detail.annual_results.hp_load_frac', 'with_heat_pump_detail.annual_results.hp_capacity_used_max'),
    test_values = (True, False)
)

,with_heat_pump_detail.annual_results.hp_load_frac,with_heat_pump_detail.annual_results.hp_capacity_used_max
doors_open_to_adjacent,,
True,0.3843,0.6223
False,0.32,0.6223


In [58]:
mod_inp = base_inp.copy()
mod_inp.bldg_model_inputs.heat_pump.max_out_5f = 25000.0
sens(
    mod_inp,
    'bldg_model_inputs.heat_pump.serves_garage', 
    ('with_heat_pump_detail.annual_results.hp_load_frac', 'with_heat_pump_detail.annual_results.hp_capacity_used_max'),
    test_values = (True, False)
)

,with_heat_pump_detail.annual_results.hp_load_frac,with_heat_pump_detail.annual_results.hp_capacity_used_max
serves_garage,,
True,0.5708,1
False,0.4998,1


In [75]:
mod_inp = base_inp.copy()
mod_inp.actual_fuel_use.secondary_fuel_units = None    # otherwise, fuel use doesn't change in base case
sens(
    mod_inp,
    'bldg_model_inputs.exist_heat_system.heating_effic', 
    ('change_detail.annual_results.fuel_units', 'change_detail.annual_results.all_kwh'),
)

,change_detail.annual_results.fuel_units,change_detail.annual_results.all_kwh
heating_effic,,
0.64,-542.7,5779
0.8,-434.2,5779
0.96,-361.8,5779


In [82]:
sens(
    base_inp,
    'bldg_model_inputs.exist_heat_system.aux_elec_use', 
    ('change_detail.annual_results.fuel_units',
     'change_detail.annual_results.all_kwh',
     'financial.bc_ratio')
)
# in base case, total kWh gets set to user-entered monthly pattern or pattern for city.

,base_case_detail.annual_results.all_kwh,change_detail.annual_results.all_kwh,financial.bc_ratio
aux_elec_use,,,
2.4,6566,7704,1.258
3,6566,7679,1.274
3.6,6566,7655,1.29


In [83]:
sens(
    base_inp,
    'bldg_model_inputs.exist_heat_system.serves_dhw', 
    ('change_detail.annual_results.fuel_units',
     'financial.bc_ratio'),
    test_values = (True, False)
)
# lower fuel associated with Space Heating when serves_dhw is True.

,change_detail.annual_results.fuel_units,financial.bc_ratio
serves_dhw,,
True,-530.3,1.142
False,-585.5,1.274


In [85]:
mod_inp = base_inp.copy()
mod_inp.bldg_model_inputs.exist_heat_system.serves_dhw = True
sens(
    mod_inp,
    'bldg_model_inputs.exist_heat_system.occupant_count', 
    ('change_detail.annual_results.fuel_units',
     'financial.bc_ratio')
)


,change_detail.annual_results.fuel_units,financial.bc_ratio
occupant_count,,
1.84,-541.6,1.168
2.3,-530.3,1.142
2.76,-518.9,1.116


In [86]:
mod_inp = base_inp.copy()
mod_inp.actual_fuel_use.secondary_fuel_units = None    # otherwise, fuel use doesn't change in base case
sens(
    mod_inp,
    'bldg_model_inputs.heat_pump.serves_garage', 
    ('change_detail.annual_results.fuel_units', ),
    test_values = (True, False)
)

,change_detail.annual_results.fuel_units
serves_garage,
True,-495.3
False,-434.2


In [87]:
mod_inp = base_inp.copy()
mod_inp.actual_fuel_use.secondary_fuel_units = None    # otherwise, fuel use doesn't change in base case
mod_inp.bldg_model_inputs.heat_pump.serves_garage = True
sens(
    mod_inp,
    'bldg_model_inputs.garage_stall_count', 
    ('change_detail.annual_results.fuel_units', ),
    test_values = (1, 2, 3)
)

,change_detail.annual_results.fuel_units
garage_stall_count,
1,-473.7
2,-495.3
3,-542.3


In [89]:
sens(
    base_inp,
    'bldg_model_inputs.indoor_heat_setpoint', 
    ('change_detail.annual_results.fuel_units',
     'financial.bc_ratio', 'with_heat_pump_detail.annual_results.cop'),
)
# I think the low B/C ratio with high fuel savings is due to the low COP at high indoor temperatures.

,change_detail.annual_results.fuel_units,financial.bc_ratio,with_heat_pump_detail.annual_results.cop
indoor_heat_setpoint,,,
56,-518.6,1.694,2.729
70,-585.5,1.274,2.421
84,-619.2,0.477,2.108


In [90]:
mod_inp = base_inp.copy()
mod_inp.actual_fuel_use.secondary_fuel_units = None    # otherwise, fuel use doesn't change in base case
sens(
    mod_inp,
    'bldg_model_inputs.insul_level', 
    ('change_detail.annual_results.fuel_units', ),
    test_values = ('wall2x4', 'wall2x6', 'wall2x6plus')
)

,change_detail.annual_results.fuel_units
insul_level,
wall2x4,-818.7
wall2x6,-651.7
wall2x6plus,-434.2


In [91]:
sens(
    base_inp,
    'heat_pump_cost.capital_cost', 
    ('financial.irr',
     'financial.bc_ratio'),
)

,financial.irr,financial.bc_ratio
capital_cost,,
3600,0.1354,1.715
4500,0.08797,1.274
5400,0.05548,1.013


In [92]:
sens(
    base_inp,
    'heat_pump_cost.rebate_amount', 
    ('financial.irr',
     'financial.bc_ratio'),
)

,financial.irr,financial.bc_ratio
rebate_amount,,
800,0.07979,1.205
1000,0.08797,1.274
1200,0.09685,1.351


In [94]:
sens(
    base_inp,
    'heat_pump_cost.heat_pump_life', 
    ('financial.irr',
     'financial.bc_ratio'),
    test_values=(10, 14, 18)
)

,financial.irr,financial.bc_ratio
heat_pump_life,,
10,0.03008,0.877
14,0.08797,1.274
18,0.1153,1.685


In [95]:
sens(
    base_inp,
    'heat_pump_cost.op_cost_chg', 
    ('financial.irr',
     'financial.bc_ratio'),
    test_values=(-50, 0, 50)
)

,financial.irr,financial.bc_ratio
op_cost_chg,,
-50,0.08797,1.274
0,0.06861,1.116
50,0.0482,0.9584


In [96]:
sens(
    base_inp,
    'heat_pump_cost.frac_financed', 
    ('financial.irr',
     'financial.bc_ratio'),
    test_values=(0.0, 0.25, 0.5)
)
# Loan interest rate of 5% is less than IRR, so IRR increases with higher levels of financing

,financial.irr,financial.bc_ratio
frac_financed,,
0,0.08797,1.274
0.25,0.09784,1.412
0.5,0.1158,1.799


In [98]:
mod_inp = base_inp.copy()
mod_inp.heat_pump_cost.frac_financed = 0.5
sens(
    mod_inp,
    'heat_pump_cost.loan_term', 
    ('financial.irr',
     'financial.bc_ratio'),
    test_values=(5, 10, 14)
)

,financial.irr,financial.bc_ratio
loan_term,,
5,0.1008,1.785
10,0.1158,1.799
14,0.1302,1.809


In [97]:
mod_inp = base_inp.copy()
mod_inp.heat_pump_cost.frac_financed = 0.5
sens(
    mod_inp,
    'heat_pump_cost.loan_interest', 
    ('financial.irr',
     'financial.bc_ratio'),
    test_values=(0.02, 0.05, 0.1)
)
# IRR did drop from base value (no financing) of 8.8% when loan interest
# rate was 10%. Otherwise IRR went up with loan interest rates less
# than base IRR.


,financial.irr,financial.bc_ratio
loan_interest,,
0.02,0.1374,2.047
0.05,0.1158,1.799
0.1,0.07914,1.345


In [100]:
sens(
    base_inp,
    'economic_inputs.customer_charge_override', 
    ('financial.irr',
     'with_heat_pump_detail.annual_results.all_elec_dol'),
    test_values=(5.0, 10.0, 20.0)
)


,financial.irr,with_heat_pump_detail.annual_results.all_elec_dol
customer_charge_override,,
5,0.08797,3207
10,0.08797,3267
20,0.08797,3387


In [101]:
sens(
    base_inp,
    'economic_inputs.co2_lbs_per_kwh_override', 
    ('misc.co2_lbs_saved',
     'misc.co2_driving_miles_saved'),
    test_values=(0.5, 1.0, 2.0)
)


,misc.co2_lbs_saved,misc.co2_driving_miles_saved
co2_lbs_per_kwh_override,,
0.5,9141,1.025e+04
1,5301,5944
2,-2378,-2666


In [102]:
sens(
    base_inp,
    'economic_inputs.fuel_price_override', 
    ('financial.irr',
     'with_heat_pump_detail.annual_results.fuel_dol'),
    test_values=(3.0, 4.0, 5.0)
)


,financial.irr,with_heat_pump_detail.annual_results.fuel_dol
fuel_price_override,,
3,0.00896,1840
4,0.2303,2453
5,0.4054,3066


In [110]:
sens(
    base_inp,
    'economic_inputs.elec_rate_forecast', 
    ('financial.bc_ratio', ),
    test_values=(0.0, 0.023, 0.05)
)


,financial.bc_ratio
elec_rate_forecast,
0,1.939
0.023,1.274
0.05,0.3306


In [111]:
sens(
    base_inp,
    'economic_inputs.elec_rate_override',
    ('financial.bc_ratio','with_heat_pump_detail.annual_results.all_elec_dol'),
    test_values=(0.15, 0.22, .3)
)


,financial.bc_ratio,with_heat_pump_detail.annual_results.all_elec_dol
elec_rate_override,,
0.15,2.992,2233
0.22,1.296,3230
0.3,-0.6423,4370


In [113]:
sens(
    base_inp,
    'economic_inputs.fuel_price_forecast',
    ('financial.bc_ratio', ),
    test_values=(0.0, 0.023, 0.033, .05)
)


,financial.bc_ratio
fuel_price_forecast,
0,0.1434
0.023,0.9012
0.033,1.274
0.05,1.976


In [114]:
sens(
    base_inp,
    'economic_inputs.sales_tax_override',
    ('financial.bc_ratio', ),
    test_values=(0.0, 0.02, 0.05)
)
# Bigger B/C cuz savings are amplified by sales tax.

,financial.bc_ratio
sales_tax_override,
0,1.274
0.02,1.296
0.05,1.33


In [115]:
sens(
    base_inp,
    'economic_inputs.discount_rate',
    ('financial.bc_ratio', ),
    test_values=(0.03, 0.0537, 0.07)
)

,financial.bc_ratio
discount_rate,
0.03,1.528
0.0537,1.274
0.07,1.132


In [116]:
sens(
    base_inp,
    'economic_inputs.inflation_rate',
    ('financial.bc_ratio', ),
    test_values=(0.0, 0.023, 0.04)
)
# Using some operating cost savings in base inputs. Inflation is applied to 
# operating cost savings, so larger value increases NPV of savings.

,financial.bc_ratio
inflation_rate,
0,1.254
0.023,1.274
0.04,1.291


In [117]:
sens(
    base_inp,
    'actual_fuel_use.secondary_fuel_units',
    ('financial.bc_ratio', ),
)

,financial.bc_ratio
secondary_fuel_units,
960,1.006
1200,1.274
1440,1.536


## Test runs to compare with Online Calculator

In [127]:
# House similar to mine in Anchorage, but heated with #1 Oil.
# For online comparison, using "Simple" Heat Pump selection.
test_inp = DM(
    {
    "bldg_name": "Test Building",
    "notes": "Notes about the analysis.",
	"bldg_model_inputs": {
		"city_id": 1,
		"heat_pump": {
			"hspf_type": "hspf",
			"hspf": 14.0,
			"max_out_5f": 13500,
			"low_temp_cutoff": 10,
			"off_months": None,
			"frac_exposed_to_hp": 0.46,
			"frac_adjacent_to_hp": 0.54,
			"doors_open_to_adjacent": True,
			"bedroom_temp_tolerance": "med",
			"serves_garage": False
		},
		"exist_heat_system": {
			"heat_fuel_id": 4,
			"heating_effic": 0.8,
			"aux_elec_use": 3.0,
			"serves_dhw": False,
			"serves_clothes_drying": False,
			"serves_cooking": False,
			"occupant_count": 2.3
		},
        "building_type": "residential",
		"bldg_floor_area": 3600,
		"garage_stall_count": 2,
		"indoor_heat_setpoint": 70,
		"insul_level": "wall2x6plus"
	},
	"heat_pump_cost": {
		"capital_cost": 4500,
		"rebate_amount": 1000,
        "heat_pump_life": 14,
		"op_cost_chg": 0,
		"frac_financed": 0.0,
		"loan_term": 10,
		"loan_interest": 0.05
	},
	"economic_inputs": {
		"utility_id": 1,
        "pce_limit": 750,
        "elec_rate_forecast": 0.023,
        "elec_rate_override": None,
        "pce_rate_override": None,
        "customer_charge_override": None,
        "co2_lbs_per_kwh_override": None,
        "fuel_price_override": None,
        "fuel_price_forecast": 0.033,
        "sales_tax_override": None,
        "discount_rate": 0.0537,
        "inflation_rate": 0.023
	},
	"actual_fuel_use": {
		"secondary_fuel_units": 1200.0,
        "annual_electric_is_just_space_heat": False,
        "electric_use_by_month": None
	}
})

In [128]:
res = post('/heat/analyze-heat-pump', test_inp)
res.pprint()
# API says 718 gallons saved and max kW for heat pump of 2.13
# Online says 712 gallons saved and max kW of 2.1 kW

{'base_case_detail': {'annual_results': {'all_elec_dol': 1546.4412073342078,
                                         'all_kw_max': 2.29368934703218,
                                         'all_kwh': 6565.84571588275,
                                         'cop': None,
                                         'fuel_dol': 3955.6523118039186,
                                         'fuel_units': 1198.6825187284603,
                                         'hp_capacity_used_max': 0.0,
                                         'hp_kw_max': 0.0,
                                         'hp_kwh': 0.0,
                                         'hp_load_frac': 0.0,
                                         'hp_load_mmbtu': 0.0,
                                         'period': 'Annual',
                                         'secondary_fuel_mmbtu': 164.7613095642643,
                                         'secondary_fuel_units': 1198.6825187284603,
                                      

In [129]:
# Juneau Home, 1,000 ft2, #1 Oil
test_inp = DM(
    {
    "bldg_name": "Test Building Juneau #1 Oil",
    "notes": "Notes about the analysis.",
	"bldg_model_inputs": {
		"city_id": 292,
		"heat_pump": {
			"hspf_type": "hspf",
			"hspf": 14.0,
			"max_out_5f": 13500,
			"low_temp_cutoff": 5,
			"off_months": None,
			"frac_exposed_to_hp": 0.6,
			"frac_adjacent_to_hp": 0.4,
			"doors_open_to_adjacent": True,
			"bedroom_temp_tolerance": "med",
			"serves_garage": False
		},
		"exist_heat_system": {
			"heat_fuel_id": 4,
			"heating_effic": 0.8,
			"aux_elec_use": 3.0,
			"serves_dhw": False,
			"serves_clothes_drying": False,
			"serves_cooking": False,
			"occupant_count": 2.3
		},
        "building_type": "residential",
		"bldg_floor_area": 1000,
		"garage_stall_count": 1,
		"indoor_heat_setpoint": 70,
		"insul_level": "wall2x6"
	},
	"heat_pump_cost": {
		"capital_cost": 4500,
		"rebate_amount": 1000,
        "heat_pump_life": 14,
		"op_cost_chg": 0,
		"frac_financed": 0.0,
		"loan_term": 10,
		"loan_interest": 0.05
	},
	"economic_inputs": {
		"utility_id": 11,
        "pce_limit": 750,
        "elec_rate_forecast": 0.023,
        "elec_rate_override": None,
        "pce_rate_override": None,
        "customer_charge_override": None,
        "co2_lbs_per_kwh_override": None,
        "fuel_price_override": None,
        "fuel_price_forecast": 0.033,
        "sales_tax_override": None,
        "discount_rate": 0.0537,
        "inflation_rate": 0.023
	},
	"actual_fuel_use": {
		"secondary_fuel_units": None,
        "annual_electric_is_just_space_heat": False,
        "electric_use_by_month": None
	}
})

In [130]:
res = post('/heat/analyze-heat-pump', test_inp)
res.pprint()
# API says 264 gallons saved, Online is 263
# API IRR is 28.8%, Online is 28.8%
# API says 81.1% of load served, Online says 81%

{'base_case_detail': {'annual_results': {'all_elec_dol': 945.5627932151003,
                                         'all_kw_max': 2.29368934703218,
                                         'all_kwh': 6565.84571588275,
                                         'cop': None,
                                         'fuel_dol': 1627.0600680720381,
                                         'fuel_units': 326.22758257083467,
                                         'hp_capacity_used_max': 0.0,
                                         'hp_kw_max': 0.0,
                                         'hp_kwh': 0.0,
                                         'hp_load_frac': 0.0,
                                         'hp_load_mmbtu': 0.0,
                                         'period': 'Annual',
                                         'secondary_fuel_mmbtu': 44.84063367952636,
                                         'secondary_fuel_units': 326.22758257083467,
                                       

In [138]:
# Juneau Home, 1,000 ft2, Electric Heat
test_inp = DM(
    {
    "bldg_name": "Test Building Juneau Electric Heat",
    "notes": "Notes about the analysis.",
	"bldg_model_inputs": {
		"city_id": 292,
		"heat_pump": {
			"hspf_type": "hspf",
			"hspf": 14.0,
			"max_out_5f": 13500,
			"low_temp_cutoff": -10,
			"off_months": None,
			"frac_exposed_to_hp": 0.6,
			"frac_adjacent_to_hp": 0.4,
			"doors_open_to_adjacent": True,
			"bedroom_temp_tolerance": "med",
			"serves_garage": False
		},
		"exist_heat_system": {
			"heat_fuel_id": 1,
			"heating_effic": 1.0,
			"aux_elec_use": 0.0,
			"serves_dhw": True,
			"serves_clothes_drying": True,
			"serves_cooking": True,
			"occupant_count": 3
		},
        "building_type": "residential",
		"bldg_floor_area": 1000,
		"garage_stall_count": 1,
		"indoor_heat_setpoint": 70,
		"insul_level": "wall2x6"
	},
	"heat_pump_cost": {
		"capital_cost": 4500,
		"rebate_amount": 1000,
        "heat_pump_life": 14,
		"op_cost_chg": 0,
		"frac_financed": 0.0,
		"loan_term": 10,
		"loan_interest": 0.05
	},
	"economic_inputs": {
		"utility_id": 11,
        "pce_limit": 750,
        "elec_rate_forecast": 0.023,
        "elec_rate_override": None,
        "pce_rate_override": None,
        "customer_charge_override": None,
        "co2_lbs_per_kwh_override": None,
        "fuel_price_override": None,
        "fuel_price_forecast": 0.033,
        "sales_tax_override": None,
        "discount_rate": 0.0537,
        "inflation_rate": 0.023
	},
	"actual_fuel_use": {
		"secondary_fuel_units": None,
        "annual_electric_is_just_space_heat": False,
        "electric_use_by_month": None
	}
})

In [139]:
res = post('/heat/analyze-heat-pump', test_inp)
res.pprint()
# API saves 5515 kWh/yr electricity, Online 5517 kWh/yr
# API IRR is 19.8%, Online is 19.8%
# API annual kW reduction: 1.34 kW, Online Feb savings was 1.4 kW
#     but Feb wasn't worst peak. Jan savings was 1.34 kW, which is worst peak.
#    (Changed low temp cutoff to -10 F so some kW savings would occur)


{'base_case_detail': {'annual_results': {'all_elec_dol': 2523.0699220575084,
                                         'all_kw_max': 5.4892202123623965,
                                         'all_kwh': 19219.449982253016,
                                         'cop': None,
                                         'fuel_dol': 0.0,
                                         'fuel_units': 0.0,
                                         'hp_capacity_used_max': 0.0,
                                         'hp_kw_max': 0.0,
                                         'hp_kwh': 0.0,
                                         'hp_load_frac': 0.0,
                                         'hp_load_mmbtu': 0.0,
                                         'period': 'Annual',
                                         'secondary_fuel_mmbtu': 0.0,
                                         'secondary_fuel_units': 0.0,
                                         'secondary_kw_max': 4.477151170231468,
             